## lcw99/t5-large-korean-text-summary 모델을 사용하여 데이터를 증강하는 코드
### ID: 새로 생성된 데이터는 ynat-v1_train_0300으로 시작한다.
### text: 원문의 의미를 유지하면서도 문장이 다소 다르게 표현된 텍스트가 생성된다.
### target: 의미를 유지하므로 라벨은 동일하다.

In [14]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from tqdm import tqdm

In [8]:
# 모델과 토크나이저 로드
model_name = "lcw99/t5-large-korean-text-summary"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [9]:
# CSV 파일 불러오기
input_file = './train.csv'  # 원본 파일 경로
output_file = 'augmented_file.csv'  # 저장할 파일 경로
data = pd.read_csv(input_file)

In [11]:
# Paraphrasing 함수 정의
def generate_paraphrase(text, model, tokenizer, max_length=50):
    input_text = f"paraphrase: {text}"
    inputs = tokenizer.encode(input_text, return_tensors="pt")
    outputs = model.generate(
        inputs,
        max_length=max_length,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        early_stopping=True
    )
    paraphrased_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return paraphrased_text

In [12]:
# Paraphrasing 적용 및 데이터 확장
new_data = []
start_id = 300  # 새롭게 추가할 ID의 시작 번호

In [13]:
for idx, row in tqdm(data.iterrows(), total=len(data), desc="Paraphrasing 진행 중"):
    original_id = row['ID']
    original_text = row['text']
    original_target = row['target']
    
    # Paraphrased text 생성
    paraphrased_text = generate_paraphrase(original_text, model, tokenizer)
    
    # 새로운 데이터 구성
    new_id = f"ynat-v1_train_{start_id:04d}"
    new_data.append([new_id, paraphrased_text, original_target])
    start_id += 1  # ID 번호 증가

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:649: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
# 원본 데이터와 Paraphrased 데이터 병합
new_data_df = pd.DataFrame(new_data, columns=['ID', 'text', 'target'])
augmented_data = pd.concat([data, new_data_df], ignore_index=True)

In [ ]:
# 예제 텍스트
news_title = "정부, 청년 실업 문제 해결 위해 새로운 정책 발표"

# Paraphrasing 수행
paraphrased_title = generate_paraphrase(news_title, model, tokenizer)
print("원문:", news_title)
print("Paraphrased:", paraphrased_title)